# Gokigen Naname

- Can be found at: [Link](https://www.janko.at/Raetsel/Gokigen-Naname/index.htm)

RULES:

1. Draw exactly one diagonal line in each cell of the grid.
2. A number in a grid dot indicates how many diagonal lines end in this grid dot.
3. **The diagonal lines must not form a closed loop.**

Algorithm:

1. Add model with Constr 1~2;
2. For any current solution, iterate all connected components and check if they form a loop. If so, use Lazy Constraint to remove these loops from the model.

![](https://www.janko.at/Raetsel/Gokigen-Naname/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Gokigen-Naname/Regeln-02.gif)


In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import gurobipy as grb # for Gurobi solver
from pyscipopt import Model, quicksum # for scip solver
import warnings 
warnings.filterwarnings("ignore")
import sys
sys.path.append("..")



In [ ]:
# Union-Find, to check loop
class UnionFind:
    def __init__(self, nodes):
        self.parent = {node: node for node in nodes}

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x == root_y:
            return False  # Loop!
        self.parent[root_y] = root_x
        return True

def find_connected_components(edges):
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)

    visited = set()
    components = []

    def dfs(node, current_component):
        visited.add(node)
        current_component.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, current_component)

    for node in graph:
        if node not in visited:
            current_component = set()
            dfs(node, current_component)
            components.append(current_component)

    grouped_edges = []
    for component in components:
        component_edges = []
        for u, v in edges:
            if u in component and v in component:
                component_edges.append((u, v))
        grouped_edges.append(component_edges)

    return grouped_edges

# Judge if loop
def check_cycles_in_components(edges):
    components = find_connected_components(edges)
    results = []

    for component_edges in components:
        nodes = set()
        for u, v in component_edges:
            nodes.add(u)
            nodes.add(v)

        uf = UnionFind(nodes)
        has_cycle = False
        for u, v in component_edges:
            if not uf.union(u, v):
                has_cycle = True
                break  

        results.append({
            'component_edges': component_edges,
            'has_cycle': has_cycle
        })

    return results

def visualize_Gokigen(m , n, arcs):
    """Visualize the whole graph ... 

    Args:
        m (_type_): _description_
        n (_type_): _description_
        arcs (_type_): _description_

    Returns:
        _type_: _description_
    """

    fig, ax = plt.subplots()


    ax.set_xticks(range(n + 2))
    ax.set_yticks(range(m + 2))
    ax.grid(True)

    ax.set_xlim(0,  n )
    ax.set_ylim(0,  m )

    ax.set_xticklabels(range(n + 2))
    ax.set_yticklabels(range(m + 2))
    ax.set_aspect('equal') 
    
    for arc in arcs:
        start, end = arc
        start_x = start % (n + 1)
        start_y = start // (n + 1)
        end_x = end % (m + 1)
        end_y = end // (m + 1)
        ax.plot([start_x, end_x], [start_y, end_y], marker='o', color='r')
        
    plt.gca().invert_yaxis()  
    plt.show()

if __name__ == "__main__":
    
    edges = [(0, 1), (1, 2), (2, 0), 
            (3, 4),                  
            (5, 6), (6, 7), (6, 8)]  
    results = check_cycles_in_components(edges)
    for i, res in enumerate(results):
        print("Is loop?:", "Yes" if res['has_cycle'] else "No")
        print("-" * 30)


In [ ]:
def readGrid(path):
    with open(f"../assets/data/GokigenNaname/problems/{path}") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        for idx, g in enumerate(res):
            res[idx] = list(map(str, g))
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("762_11x11.txt")
    print(m, n)
    for row in grid:
        print(row)


## SCIP solver (Verified!)

In [ ]:
def GokigenNanameSolverSCIP(m, n, grid):
    GokigenNaname = Model("GokigenNaname")
    x = {}
    keys = []
    # 严格保持Gurobi的变量创建逻辑
    for i in range(m):
        for j in range(n):
            current_node = i * n + j
            # 左下斜线（与Gurobi完全一致的条件判断）
            if i + 1 < m and j - 1 >= 0:
                x[i * n + j, (i + 1) * n + j - 1] = GokigenNaname.addVar(
                    vtype='B', 
                    obj = 1,
                    name=f"x[ {i * n + j}, {(i + 1) * n + j - 1}]"
                )
                keys.append([i * n + j, (i + 1) * n + j - 1])
            # 右下斜线
            if i + 1 < m and j + 1 < n:
                x[i * n + j, (i + 1) * n + j + 1] = GokigenNaname.addVar(
                    vtype='B',
                    obj = 1,
                    name=f"x[ {i * n + j}, {(i + 1) * n + j + 1}]"
                )
                keys.append([i * n + j, (i + 1) * n + j + 1])
    
    # 数字约束（保持原始索引计算方式）
    for i in range(m):
        for j in range(n):
            if grid[i][j].isdigit():
                temp_list = []
                current_node = i * n + j
                # 上左节点
                if i - 1 >= 0 and j - 1 >= 0:
                    prev_node = (i - 1) * n + (j - 1)
                    if (prev_node, current_node) in x:
                        temp_list.append(x[(prev_node, current_node)])
                # 上右节点
                if i - 1 >= 0 and j + 1 < n:
                    prev_node = (i - 1) * n + (j + 1)
                    if (prev_node, current_node) in x:
                        temp_list.append(x[(prev_node, current_node)])
                # 下右节点
                if i + 1 < m and j + 1 < n:
                    next_node = (i + 1) * n + (j + 1)
                    if (current_node, next_node) in x:
                        temp_list.append(x[(current_node, next_node)])
                # 下左节点
                if i + 1 < m and j - 1 >= 0:
                    next_node = (i + 1) * n + (j - 1)
                    if (current_node, next_node) in x:
                        temp_list.append(x[(current_node, next_node)])
                
                if temp_list:
                    GokigenNaname.addCons(
                        quicksum(temp_list) == int(grid[i][j]),
                        name=f"Num_{i}_{j}"
                    )
    

    for i in range(m - 1):
        for j in range(n - 1):

            
            
            GokigenNaname.addCons(
                quicksum([x[i * m + j, (i + 1) * m + j + 1], x[i * m + j + 1, (i + 1) * m + j]]) == 1,
                name=f"NoConflict_{i}_{j}"
            )
    

    GokigenNaname.setObjective(quicksum(x[a, b] for (a, b) in keys), sense="minimize")
    GokigenNaname.hideOutput()
    
    while True:
        GokigenNaname.optimize()
        status = GokigenNaname.getStatus()
        cur_arcs = []
        for (a, b) in x.keys():
            if GokigenNaname.getVal(x[a, b]) > 1e-1:
                cur_arcs.append((a, b))
        temp_check = check_cycles_in_components(cur_arcs)
        new_constraints = False
        for cnt_component in temp_check:
            if cnt_component['has_cycle']:
                if not new_constraints:
                    GokigenNaname.freeTransform()
                    new_constraints = True
                edges = cnt_component['component_edges']
                GokigenNaname.addCons(
                    quicksum(x[i,j] for (i,j) in edges) <= len(edges) - 1)
        if not new_constraints:
            break
    
    arcs = []
    for (a, b) in keys:
        if (GokigenNaname.getVal(x[a, b]) > 0.5):
            arcs.append([a, b])
    # sol = model.getBestSol()
    return arcs
    
if __name__ == "__main__":

    m, n, grid = readGrid("221_11x11.txt")
    arcs = GokigenNanameSolverSCIP(m, n, grid)
    visualize_Gokigen(m - 1, n - 1, arcs)


## Gurobi solver (Verified!)

In [ ]:
def GokigenNanameSolver(m, n, grid):
    x = dict()
    GokigenNaname = grb.Model("GokigenNaname")
    
    for i in range(m):
        for j in range(n):
            if i + 1 < m and j - 1 >= 0: 
                x[i * n + j, (i + 1) * n + j - 1] = GokigenNaname.addVar(
                    vtype=grb.GRB.BINARY,
                    obj = 1,
                    name = f"x[ {i * n + j}, {(i + 1) * n + j - 1}]"
                )
            if i + 1 < m and j + 1 < n:
                x[i * n + j, (i + 1) * n + j + 1] = GokigenNaname.addVar(
                    vtype=grb.GRB.BINARY,
                    obj = 1,
                    name = f"x[ {i * n + j}, {(i + 1) * n + j + 1}]"
                )
    for i in range(m):
        for j in range(n):
            if grid[i][j] in "1234567890":
                # add constrs here
                temp_list = []
                if i - 1 >= 0 and j - 1 >= 0:
                    temp_list.append(x[(i - 1) * n + j - 1, i * n + j])
                if i - 1 >= 0 and j + 1 < n:
                    temp_list.append(x[(i - 1) * n + j + 1, i * n + j])
                if i + 1 < m and j + 1 < n:
                    temp_list.append(x[i * n + j, (i + 1) * n + j + 1])
                if i + 1 < m and j - 1 >= 0:
                    temp_list.append(x[i * n + j, (i + 1) * n + j - 1])
                if len(temp_list) > 0:
                    GokigenNaname.addConstr(grb.quicksum(temp_list) == int(grid[i][j]), name = f"Num_{i}_{j}")

    for i in range(m - 1):
        for j in range(n - 1):
            GokigenNaname.addConstr(grb.quicksum([x[i * m + j, (i + 1) * m + j + 1], x[i * m + j + 1, (i + 1) * m + j]]) == 1, name = f"NoConflict_{i}_{j}")
    

    GokigenNaname.modelSense = grb.GRB.MINIMIZE
    GokigenNaname.Params.lazyConstraints = 1
    GokigenNaname.update()
    GokigenNaname._x = x
    
    def subloopelim(model, where):
        if (where == grb.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            cur_arcs = []
            for (i, j) in x.keys():
                if x_sol[i, j] > 1e-1:
                    cur_arcs.append((i, j))
            temp_check = check_cycles_in_components(cur_arcs)
            
            for cnt_component in temp_check:
                if cnt_component['has_cycle']:
                    # print(f"Add elim Routes: {cnt_component['component_edges']}")
                    model.cbLazy(
                        grb.quicksum(x[i, j] for (i, j) in cnt_component['component_edges']) \
                         <= len(cnt_component['component_edges']) - 1)

    GokigenNaname._x = x
    GokigenNaname.optimize(subloopelim)
    ofv = None
    seq = []
    arcs = []
    solType = None
    gap = None
    lb = None
    ub = None
    runtime = None
    if (GokigenNaname.status == grb.GRB.status.OPTIMAL):
        solType = 'IP_Optimal'
        ofv = GokigenNaname.getObjective().getValue()
        print(ofv)
        for i, j in x:
            if (x[i, j].x > 0.5):
                arcs.append([i, j])
        gap = 0
        lb = ofv
        ub = ofv
        runtime = GokigenNaname.Runtime
    elif (GokigenNaname.status == grb.GRB.status.TIME_LIMIT):
        solType = 'IP_TimeLimit'
        ofv = None
        seq = []
        gap = GokigenNaname.MIPGap
        lb = GokigenNaname.ObjBoundC
        ub = GokigenNaname.ObjVal
        runtime = GokigenNaname.Runtime
    
    return arcs


if __name__ == "__main__":
    m, n, grid = readGrid("762_11x11.txt")
    arcs = GokigenNanameSolver(m, n, grid)
    visualize_Gokigen(m - 1, n - 1, arcs)